***GENERATED CODE FOR datapreparation PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ParcelID', 'transformation_label': 'String Indexer'}], 'feature': 'ParcelID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '3352', 'mean': '', 'stddev': '', 'min': '104-001-013', 'max': '999-999-424', 'missing': '0', 'distinct': '2723'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ParcelID'}, {'feature_label': 'ParcelID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ParcelID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ApartmentUnitNumber', 'transformation_label': 'String Indexer'}], 'feature': 'ApartmentUnitNumber', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3352', 'mean': '144.06', 'stddev': '224.29', 'min': '#4', 'max': 'WEST', 'missing': '0', 'distinct': '522'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ApartmentUnitNumber'}, {'feature_label': 'ApartmentUnitNumber', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ApartmentUnitNumber')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StreetNameAndWay', 'transformation_label': 'String Indexer'}], 'feature': 'StreetNameAndWay', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3352', 'mean': '', 'stddev': '', 'min': 'ADAMS ST', 'max': 'ZION ST', 'missing': '0', 'distinct': '386'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StreetNameAndWay'}, {'feature_label': 'StreetNameAndWay', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StreetNameAndWay')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OwnerLastName', 'transformation_label': 'String Indexer'}], 'feature': 'OwnerLastName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3352', 'mean': '', 'stddev': '', 'min': '1000 CAPITOL LLC', 'max': 'ZUBAIR', 'missing': '0', 'distinct': '1650'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OwnerLastName'}, {'feature_label': 'OwnerLastName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OwnerLastName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OwnerFirstName', 'transformation_label': 'String Indexer'}], 'feature': 'OwnerFirstName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3352', 'mean': '', 'stddev': '', 'min': 'A ROBERT', 'max': 'ZION', 'missing': '0', 'distinct': '1195'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OwnerFirstName'}, {'feature_label': 'OwnerFirstName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OwnerFirstName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryGrantor', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryGrantor', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3352', 'mean': '', 'stddev': '', 'min': '1047 ALBANY AVE LLC', 'max': 'ZIMMERMAN ERICH', 'missing': '0', 'distinct': '2470'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryGrantor'}, {'feature_label': 'PrimaryGrantor', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryGrantor')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SaleDate', 'transformation_label': 'String Indexer'}], 'feature': 'SaleDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3352', 'mean': '', 'stddev': '', 'min': '1/10/2022', 'max': '9/9/2022', 'missing': '0', 'distinct': '475'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SaleDate'}, {'feature_label': 'SaleDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SaleDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LegalReference', 'transformation_label': 'String Indexer'}], 'feature': 'LegalReference', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3352', 'mean': '7967.4', 'stddev': '40.2', 'min': '07452-0133', 'max': '8011', 'missing': '0', 'distinct': '2788'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LegalReference'}, {'feature_label': 'LegalReference', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LegalReference')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AssrLandUse', 'transformation_label': 'String Indexer'}], 'feature': 'AssrLandUse', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3352', 'mean': '', 'stddev': '', 'min': 'APT CRDA', 'max': 'TWO FAMILY', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AssrLandUse'}, {'feature_label': 'AssrLandUse', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AssrLandUse')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run datapreparationHooks.ipynb
try:
	#sourcePreExecutionHook()

	realestatesalesdays = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Real_Estate_Sales_730_Days.csv', 'filename': 'Real_Estate_Sales_730_Days.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run datapreparationHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofedataset = TransformationMain.run(realestatesalesdays,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "PropertyID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3352", "mean": "11244.2", "stddev": "6474.68", "min": "9", "max": "29427", "missing": "0"}, "updatedLabel": "PropertyID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "xrCompositeLandUseID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3352", "mean": "9.92", "stddev": "47.4", "min": "3", "max": "455", "missing": "0"}, "updatedLabel": "xrCompositeLandUseID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "xrBuildingTypeID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3352", "mean": "10.47", "stddev": "4.97", "min": "0", "max": "30", "missing": "0"}, "updatedLabel": "xrBuildingTypeID"}, {"transformationsData": [{"feature_label": "ParcelID", "transformation_label": "String Indexer"}], "feature": "ParcelID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3352", "mean": "", "stddev": "", "min": "104-001-013", "max": "999-999-424", "missing": "0", "distinct": "2723"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ParcelID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LocationStartNumber", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3352", "mean": "148.83", "stddev": "235.74", "min": "1", "max": "2718", "missing": "0"}, "updatedLabel": "LocationStartNumber"}, {"transformationsData": [{"feature_label": "ApartmentUnitNumber", "transformation_label": "String Indexer"}], "feature": "ApartmentUnitNumber", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3352", "mean": "144.06", "stddev": "224.29", "min": "#4", "max": "WEST", "missing": "0", "distinct": "522"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ApartmentUnitNumber"}, {"transformationsData": [{"feature_label": "StreetNameAndWay", "transformation_label": "String Indexer"}], "feature": "StreetNameAndWay", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3352", "mean": "", "stddev": "", "min": "ADAMS ST", "max": "ZION ST", "missing": "0", "distinct": "386"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StreetNameAndWay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "xrPrimaryNeighborhoodID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3352", "mean": "139.09", "stddev": "100.07", "min": "1", "max": "492", "missing": "0"}, "updatedLabel": "xrPrimaryNeighborhoodID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LandSF", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3352", "mean": "7769.62", "stddev": "5004.73", "min": "0", "max": "156200", "missing": "0"}, "updatedLabel": "LandSF"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TotalFinishedArea", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3352", "mean": "2215.51", "stddev": "5207.34", "min": "399.0", "max": "293441.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TotalFinishedArea"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "LivingUnits", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3352", "mean": "1.71", "stddev": "4.44", "min": "0", "max": "252", "missing": "0"}, "updatedLabel": "LivingUnits"}, {"transformationsData": [{"feature_label": "OwnerLastName", "transformation_label": "String Indexer"}], "feature": "OwnerLastName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3352", "mean": "", "stddev": "", "min": "1000 CAPITOL LLC", "max": "ZUBAIR", "missing": "0", "distinct": "1650"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OwnerLastName"}, {"transformationsData": [{"feature_label": "OwnerFirstName", "transformation_label": "String Indexer"}], "feature": "OwnerFirstName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3352", "mean": "", "stddev": "", "min": "A ROBERT", "max": "ZION", "missing": "0", "distinct": "1195"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OwnerFirstName"}, {"transformationsData": [{"feature_label": "PrimaryGrantor", "transformation_label": "String Indexer"}], "feature": "PrimaryGrantor", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3352", "mean": "", "stddev": "", "min": "1047 ALBANY AVE LLC", "max": "ZIMMERMAN ERICH", "missing": "0", "distinct": "2470"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryGrantor"}, {"transformationsData": [{"feature_label": "SaleDate", "transformation_label": "String Indexer"}], "feature": "SaleDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3352", "mean": "", "stddev": "", "min": "1/10/2022", "max": "9/9/2022", "missing": "0", "distinct": "475"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SaleDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SalePrice", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3352", "mean": "1258304.05", "stddev": "5323050.67", "min": "0", "max": "60500000", "missing": "0"}, "updatedLabel": "SalePrice"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TotalAppraisedValue", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3352", "mean": "186682.62", "stddev": "405148.74", "min": "0", "max": "22640100", "missing": "0"}, "updatedLabel": "TotalAppraisedValue"}, {"transformationsData": [{"feature_label": "LegalReference", "transformation_label": "String Indexer"}], "feature": "LegalReference", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3352", "mean": "7967.4", "stddev": "40.2", "min": "07452-0133", "max": "8011", "missing": "0", "distinct": "2788"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LegalReference"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "xrSalesValidityID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3352", "mean": "11.03", "stddev": "9.97", "min": "0", "max": "27", "missing": "0"}, "updatedLabel": "xrSalesValidityID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "xrDeedID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3352", "mean": "9.2", "stddev": "2.44", "min": "0", "max": "19", "missing": "0"}, "updatedLabel": "xrDeedID"}, {"transformationsData": [{"feature_label": "AssrLandUse", "transformation_label": "String Indexer"}], "feature": "AssrLandUse", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3352", "mean": "", "stddev": "", "min": "APT CRDA", "max": "TWO FAMILY", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AssrLandUse"}]}))

	#transformationPostExecutionHook(autofedataset)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run datapreparationHooks.ipynb
try:
	#sinkPreExecutionHook()

	realestatesalesdaysoutput = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/65c951f4f76b44cd9a30e2c7/part-00000-dc6a1fcb-25b9-44e5-bf77-88163e019fca-c000.csv', 'samplecount': 413, 'originalcount': 4088, 'orignalKey': None, 'pathOnly': '/Real Estate Sales 730 Days', 'project_id': '65c1c724082b770462799d26', 'parent_id': '65c1c724082b770462799d26', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e35d', 'field': 'PropertyID', 'alias': 'PropertyID', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e35e', 'field': 'xrCompositeLandUseID', 'alias': 'xrCompositeLandUseID', 'type': 'numeric', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e35f', 'field': 'xrBuildingTypeID', 'alias': 'xrBuildingTypeID', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e360', 'field': 'LocationStartNumber', 'alias': 'LocationStartNumber', 'type': 'numeric', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e361', 'field': 'xrPrimaryNeighborhoodID', 'alias': 'xrPrimaryNeighborhoodID', 'type': 'numeric', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e362', 'field': 'LandSF', 'alias': 'LandSF', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e363', 'field': 'TotalFinishedArea', 'alias': 'TotalFinishedArea', 'type': 'real', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e364', 'field': 'LivingUnits', 'alias': 'LivingUnits', 'type': 'numeric', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e365', 'field': 'SalePrice', 'alias': 'SalePrice', 'type': 'numeric', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e366', 'field': 'TotalAppraisedValue', 'alias': 'TotalAppraisedValue', 'type': 'numeric', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e367', 'field': 'xrSalesValidityID', 'alias': 'xrSalesValidityID', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e368', 'field': 'xrDeedID', 'alias': 'xrDeedID', 'type': 'numeric', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e369', 'field': 'ParcelID_stringindexer', 'alias': 'ParcelID_stringindexer', 'type': 'real', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e36a', 'field': 'ApartmentUnitNumber_stringindexer', 'alias': 'ApartmentUnitNumber_stringindexer', 'type': 'real', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e36b', 'field': 'StreetNameAndWay_stringindexer', 'alias': 'StreetNameAndWay_stringindexer', 'type': 'real', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e36c', 'field': 'OwnerLastName_stringindexer', 'alias': 'OwnerLastName_stringindexer', 'type': 'real', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e36d', 'field': 'OwnerFirstName_stringindexer', 'alias': 'OwnerFirstName_stringindexer', 'type': 'real', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e36e', 'field': 'PrimaryGrantor_stringindexer', 'alias': 'PrimaryGrantor_stringindexer', 'type': 'real', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e36f', 'field': 'SaleDate_stringindexer', 'alias': 'SaleDate_stringindexer', 'type': 'real', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e370', 'field': 'LegalReference_stringindexer', 'alias': 'LegalReference_stringindexer', 'type': 'real', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e371', 'field': 'AssrLandUse_stringindexer', 'alias': 'AssrLandUse_stringindexer', 'type': 'real', 'position': '20'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e348', 'field': 'PropertyID', 'alias': 'PropertyID', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e349', 'field': 'xrCompositeLandUseID', 'alias': 'xrCompositeLandUseID', 'type': 'numeric', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e34a', 'field': 'xrBuildingTypeID', 'alias': 'xrBuildingTypeID', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e34b', 'field': 'ParcelID', 'alias': 'ParcelID', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e34c', 'field': 'LocationStartNumber', 'alias': 'LocationStartNumber', 'type': 'numeric', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e34d', 'field': 'ApartmentUnitNumber', 'alias': 'ApartmentUnitNumber', 'type': 'String', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e34e', 'field': 'StreetNameAndWay', 'alias': 'StreetNameAndWay', 'type': 'String', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e34f', 'field': 'xrPrimaryNeighborhoodID', 'alias': 'xrPrimaryNeighborhoodID', 'type': 'numeric', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e350', 'field': 'LandSF', 'alias': 'LandSF', 'type': 'real', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e351', 'field': 'TotalFinishedArea', 'alias': 'TotalFinishedArea', 'type': 'real', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e352', 'field': 'LivingUnits', 'alias': 'LivingUnits', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e353', 'field': 'OwnerLastName', 'alias': 'OwnerLastName', 'type': 'String', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e354', 'field': 'OwnerFirstName', 'alias': 'OwnerFirstName', 'type': 'String', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e355', 'field': 'PrimaryGrantor', 'alias': 'PrimaryGrantor', 'type': 'String', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e356', 'field': 'SaleDate', 'alias': 'SaleDate', 'type': 'date', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e357', 'field': 'SalePrice', 'alias': 'SalePrice', 'type': 'numeric', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e358', 'field': 'TotalAppraisedValue', 'alias': 'TotalAppraisedValue', 'type': 'numeric', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e359', 'field': 'LegalReference', 'alias': 'LegalReference', 'type': 'String', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e35a', 'field': 'xrSalesValidityID', 'alias': 'xrSalesValidityID', 'type': 'numeric', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e35b', 'field': 'xrDeedID', 'alias': 'xrDeedID', 'type': 'numeric', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c95379f76b44cd9a30e35c', 'field': 'AssrLandUse', 'alias': 'AssrLandUse', 'type': 'String', 'position': '20'}], 'server': 'https://ml.colaberry.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'AutoFECleanedDataset.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-b478d759-af96-46ef-85d2-66c3fe8a929f-c000.csv', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
